In [ ]:
%run "P:\Flat-Field-Correction\Functions.py"

In [ ]:
# Import Images
data = skimage.io.ImageCollection('*.tif', conserve_memory=False, load_func=imread_rgb)
ZO1 = [image for image in data]

In [ ]:
# Sum image intensities and select images with the greatest amount of signal
image_sum_stat = pd.Series([np.sum(image) for image in ZO1])
image_sum_percentile = image_sum_stat.quantile(.90)

blurred_percent_data = []

# gaussian blur images to mimic light distribution for each channel
for index, image in tqdm(enumerate(ZO1)):
    if np.sum(image) > image_sum_percentile:
        image_blur = skimage.filters.gaussian(image, sigma=30)
        blurred_percent_data.append(image_blur)
    else:
        pass

burred_data_array = skimage.io.concatenate_images(blurred_percent_data)

In [ ]:
# Calulate reference image by taking median intensity of each pixel between the image set
reference = np.median(burred_data_array, axis=0)
plt.imshow(reference, cmap='gray')

In [ ]:
# Export 8 bit and 16 bit refernce images
%cd "Output"
ref_8 = skimage.img_as_ubyte(reference)
ref_16 = skimage.img_as_uint(reference)
skimage.io.imsave('ref_16.tif', ref_16)
skimage.io.imsave('ref_8.tif', ref_8)